In [ ]:
import argparse
import importlib
import torch
import torch.nn as nn
from defocus.model import Model

In [ ]:
argv = ['--batch_size', '4', '--num_workers', '4']

parser = argparse.ArgumentParser(description='It is time for more... experiments.')
parser.add_argument('--batch_size', type=int, default=8, help='input batch size for training')
parser.add_argument('--num_workers', type=int, default=6, help='the number of dataloader workers')
parser.add_argument('--distributed', action='store_true', help='blurb')
parser.add_argument('--world_size', default=1, type=int,
                    help='number of nodes for distributed training')

args = parser.parse_args(argv)

In [ ]:
model_name = 'MSResNet'
architecture = importlib.import_module('defocus.architecture.' + model_name)
training = importlib.import_module('defocus.trainers.' + model_name)

In [ ]:
G = architecture.Generator()
D = architecture.Discriminator()

In [ ]:
model = Model()
model.G = G
model.D = D
# model.use_perceptual()
model.set_G_optimizer('AdamP')
model.set_D_optimizer('AdamP')

In [ ]:
model.set_resconstruction_loss(loss_functions=[nn.L1Loss()], 
                               weights=[1.0])
model.set_adversarial_loss(loss_functions=[nn.BCEWithLogitsLoss()],
                           weights=[1.0])

In [ ]:
data = importlib.import_module('defocus.data.' + model_name)
train_dataset = data.Dataset(root_folder='/storage/projects/all_datasets/GOPRO/train/', 
                             image_pair_list='/storage/projects/all_datasets/GOPRO/train/train_image_pair_list.txt',
                            )
validation_dataset = data.Dataset(root_folder='/storage/projects/all_datasets/GOPRO/train/', 
                                  image_pair_list='/storage/projects/all_datasets/GOPRO/train/val_image_pair_list.txt',
                                 )

In [ ]:
trainer = training.Trainer(model, 
                           train_dataset, validation_dataset,
                           batch_size=args.batch_size,
                           num_workers=args.num_workers
                          )

In [ ]:
trainer.train(epoch=0)